# Image Generation Examples

The Amazon Nova 2 Omni model can generate an image from a simple text prompt or from a combination of text and a single input image. All image generation is achieved through text prompting alone - no complex image generation parameters needed!

## Supported Resolutions and Aspect Ratios

**Text-to-Image:**
- Maximum output size: 4,194,304 pixels (4 megapixels)
- Default aspect ratio: 16:9
- Landscape: 2:1 (2880x1440), 16:9 (2704x1520), 3:2 (2496x1664), 4:3 (2352x1760)
- Square: 1:1 (2048x2048)
- Portrait: 1:2 (1440x2880), 9:16 (1520x2704), 2:3 (1664x2496), 3:4 (1760x2352)

**Image Editing:**
- Maximum output size: 1,048,576 pixels (1 megapixel)
- Output matches input aspect ratio unless explicitly requested otherwise

---

### Helper Functions

Run the cell below to establish some helper functions used by the examples.

In [ ]:
import json
import os

import boto3
from botocore.config import Config
from botocore.exceptions import ClientError

import nova_notebook_utils
import nova_utils

MODEL_ID = "us.amazon.nova-2-omni-v1:0"
REGION_ID = "us-west-2"

# Configure the Bedrock Runtime client with an extended read timeout.
config = Config(
    read_timeout=60 * 5,
    retries={"max_attempts": 0},
)
bedrock_runtime = boto3.client(
    service_name="bedrock-runtime",
    region_name=REGION_ID,
    config=config,
)

## Text-to-Image Generation

Generate images from text prompts. At minimum, include a phrase that makes it clear you want an image generated.

---

### Example 1: Basic Text-to-Image

Simple prompts work well with Nova 2 Omni.

In [ ]:
new_message = {
    "role": "user",
    "content": [
        {"text": "Create an image of a cozy campfire at night"},
    ],
}

request = {
    "modelId": MODEL_ID,
    "messages": [new_message],
}

# == Uncomment below to include a system prompt ==
# request["system"] = [{"text": "Always generate images using 1:1 aspect ratio"}]

# == Uncomment below to enable reasoning ==
# request["additionalModelRequestFields"] = {
#     "reasoningConfig": {
#         "type": "enabled",
#         "maxReasoningEffort": "low",  # "low" | "medium" | "high"
#     }
# }

# == Uncomment below to set your own inference params ==
# request["inferenceConfig"] = {"temperature": 0, "topP": 1, "maxTokens": 10000}

try:
    response = nova_utils.converse(
        bedrock_runtime=bedrock_runtime, request=request, output_dir="output"
    )
    nova_notebook_utils.render_response(response)
except ClientError as e:
    print(f"Error: {json.dumps(e.response, indent=4)}")
except Exception as e:
    print(f"Error: {e}")

---

### Example 2: Setting Aspect Ratio

Request specific aspect ratios by mentioning them in your prompt. Default is 16:9.

In [ ]:
new_message = {
    "role": "user",
    "content": [
        {"text": "Create an image of a pyramid of ice cream scoops on a plain yellow background. Make it square."},
    ],
}

request = {"modelId": MODEL_ID, "messages": [new_message]}

try:
    response = nova_utils.converse(bedrock_runtime=bedrock_runtime, request=request, output_dir="output")
    nova_notebook_utils.render_response(response)
except ClientError as e:
    print(f"Error: {json.dumps(e.response, indent=4)}")

---

### Example 3: Setting Visual Style

Dictate the visual style by describing it in your prompt.

In [ ]:
new_message = {
    "role": "user",
    "content": [
        {"text": "Create a whimsical storybook illustration of a fox character wearing a backpack"},
    ],
}

request = {"modelId": MODEL_ID, "messages": [new_message]}

try:
    response = nova_utils.converse(bedrock_runtime=bedrock_runtime, request=request, output_dir="output")
    nova_notebook_utils.render_response(response)
except ClientError as e:
    print(f"Error: {json.dumps(e.response, indent=4)}")

---

### Example 4: Detailed Creative Control

Be more descriptive about subject, environment, and action for greater control.

In [ ]:
new_message = {
    "role": "user",
    "content": [
        {"text": "Create an image of a young girl detective. She should be wearing a beret and a checkered shirt. She is in an alley examining the ground with a magnifying glass where some dark footprints are visible. It's daytime with stark shadows. The image should be in a colorful graphic novel style with strong line work."},
    ],
}

request = {"modelId": MODEL_ID, "messages": [new_message]}

try:
    response = nova_utils.converse(bedrock_runtime=bedrock_runtime, request=request, output_dir="output")
    nova_notebook_utils.render_response(response)
except ClientError as e:
    print(f"Error: {json.dumps(e.response, indent=4)}")

---

### Example 5: Generating Text in Images

For single line text, wrap it in double quotes. For multiline text, clearly set it apart.

In [ ]:
new_message = {
    "role": "user",
    "content": [
        {"text": """Generate a medium closeup photo of the front window of a charming restaurant. The words "Trattoria Nova" can be seen on the glass, written in a gold calligraphy font that conveys elegance"""},
    ],
}

request = {"modelId": MODEL_ID, "messages": [new_message]}

try:
    response = nova_utils.converse(bedrock_runtime=bedrock_runtime, request=request, output_dir="output")
    nova_notebook_utils.render_response(response)
except ClientError as e:
    print(f"Error: {json.dumps(e.response, indent=4)}")

---

## Image Editing with Input Images

Nova 2 Omni allows you to include a single image as input for image generation. You can edit the image or use it as a reference.

---

### Example 6: Adding/Removing/Replacing Items

Edit images by adding, removing, or replacing items.

In [ ]:
input_image_path = "media/man_crossing_street.png"
image_bytes, image_format = nova_utils.load_image_as_bytes(input_image_path)

new_message = {
    "role": "user",
    "content": [
        {"image": {"format": image_format, "source": {"bytes": image_bytes}}},
        {"text": "Add a cup of tea to the scene"},
    ],
}

request = {"modelId": MODEL_ID, "messages": [new_message]}

try:
    response = nova_utils.converse(bedrock_runtime=bedrock_runtime, request=request, output_dir="output")
    nova_notebook_utils.render_response(response)
except ClientError as e:
    print(f"Error: {json.dumps(e.response, indent=4)}")

---

### Example 7: Changing Background

Replace or modify the background of an image.

In [ ]:
input_image_path = "media/man_crossing_street.png"
image_bytes, image_format = nova_utils.load_image_as_bytes(input_image_path)

new_message = {
    "role": "user",
    "content": [
        {"image": {"format": image_format, "source": {"bytes": image_bytes}}},
        {"text": "Replace the background with a view of the mountains"},
    ],
}

request = {"modelId": MODEL_ID, "messages": [new_message]}

try:
    response = nova_utils.converse(bedrock_runtime=bedrock_runtime, request=request, output_dir="output")
    nova_notebook_utils.render_response(response)
except ClientError as e:
    print(f"Error: {json.dumps(e.response, indent=4)}")

---

### Example 8: Restyling an Image

Change the artistic style of an image.

In [ ]:
input_image_path = "media/man_crossing_street.png"
image_bytes, image_format = nova_utils.load_image_as_bytes(input_image_path)

new_message = {
    "role": "user",
    "content": [
        {"image": {"format": image_format, "source": {"bytes": image_bytes}}},
        {"text": "Change the style of the image to a 3D animated film"},
    ],
}

request = {"modelId": MODEL_ID, "messages": [new_message]}

try:
    response = nova_utils.converse(bedrock_runtime=bedrock_runtime, request=request, output_dir="output")
    nova_notebook_utils.render_response(response)
except ClientError as e:
    print(f"Error: {json.dumps(e.response, indent=4)}")

---

### Example 9: Changing Aspect Ratio

Expand or change the aspect ratio of an existing image.

In [ ]:
input_image_path = "media/man_crossing_street.png"
image_bytes, image_format = nova_utils.load_image_as_bytes(input_image_path)

new_message = {
    "role": "user",
    "content": [
        {"image": {"format": image_format, "source": {"bytes": image_bytes}}},
        {"text": "Change the aspect ratio to square"},
    ],
}

request = {"modelId": MODEL_ID, "messages": [new_message]}

try:
    response = nova_utils.converse(bedrock_runtime=bedrock_runtime, request=request, output_dir="output")
    nova_notebook_utils.render_response(response)
except ClientError as e:
    print(f"Error: {json.dumps(e.response, indent=4)}")

---

### Example 10: Extracting Elements

Extract specific elements from an image.

In [ ]:
input_image_path = "media/man_crossing_street.png"
image_bytes, image_format = nova_utils.load_image_as_bytes(input_image_path)

new_message = {
    "role": "user",
    "content": [
        {"image": {"format": image_format, "source": {"bytes": image_bytes}}},
        {"text": "Extract the foreground object in this image and place it on a white background"},
    ],
}

request = {"modelId": MODEL_ID, "messages": [new_message]}

try:
    response = nova_utils.converse(bedrock_runtime=bedrock_runtime, request=request, output_dir="output")
    nova_notebook_utils.render_response(response)
except ClientError as e:
    print(f"Error: {json.dumps(e.response, indent=4)}")